In [35]:
import pandas as pd

# Read in CSV File
season_box = pd.read_csv('data/box_scores/season_box_scores_20202021.csv', converters={'GAME_ID': str, 'TEAM_ABBREVIATION': str}, index_col=False)
season_games = pd.read_csv(
    'data/games/season_games_20202021.csv', converters={'Game_ID': str, 'TEAM_ABBREVIATION': str}, index_col=False)\

# Define columns to keep from games dataframe
col_keep = ['W_PCT', 'FG_PCT', 'FG3_PCT',
            'FT_PCT', 'OREB', 'AST', 'STL', 'BLK', 'TOV']


In [36]:
# Group by GAME_ID in season_20_21_box and keep only first two rows
season_box = season_box.groupby(
    'GAME_ID').head(2).reset_index(drop=True)

In [37]:
# Remove columns not needed
season_box.drop(['LEAGUE_ID', 'TEAM_ID', 'TEAM_CITY', 'GAME_DATE_EST', 'GAME_SEQUENCE',
                 'TEAM_ID.1', 'TEAM_ABBREVIATION.1', 'TEAM_CITY_NAME', 'TEAM_NICKNAME', 'TEAM_WINS_LOSSES', 'TEAM_TURNOVERS', 'TOTAL_TURNOVERS'], axis=1, inplace=True)


In [38]:
season_box.head()

,TEAM_ABBREVIATION,PTS_PAINT,PTS_2ND_CHANCE,PTS_FB,LARGEST_LEAD,LEAD_CHANGES,TIMES_TIED,TEAM_REBOUNDS,PTS_OFF_TO,GAME_ID,...,PTS_OT2,PTS_OT3,PTS_OT4,PTS_OT5,PTS_OT6,PTS_OT7,PTS_OT8,PTS_OT9,PTS_OT10,PTS
0,HOU,24,13,12,3,7,4,8,26,0022001066,...,0,0,0,0,0,0,0,0,0,95
1,ATL,70,23,11,31,7,4,7,14,0022001066,...,0,0,0,0,0,0,0,0,0,124
2,ORL,38,21,8,3,1,0,11,17,0022001049,...,0,0,0,0,0,0,0,0,0,116
3,ATL,42,10,12,30,1,0,9,9,0022001049,...,0,0,0,0,0,0,0,0,0,93
4,WAS,58,15,13,13,3,7,9,7,0022001042,...,0,0,0,0,0,0,0,0,0,120


In [46]:
# Get unique values of Game_ID column in season_20_21_games
game_ids = season_games['Game_ID'].unique()

season_box_ext = pd.DataFrame(columns=season_box.columns)

for id in game_ids:
    # Get all rows with id in Game_ID column
    df_game = season_games[season_games['Game_ID'] == id]
    # Split MATCHUP column string at 'vs.' and strip whitespace
    matchup_split = df_game.iloc[0]['MATCHUP'].split(' ')
    team_one = matchup_split[0].strip()
    team_two = matchup_split[2].strip()
    # Drop columns not in col_keep
    df_game = df_game[col_keep]
    # Make new dataframe out of both rows
    df_team_one = df_game.iloc[0:1]
    df_team_two = df_game.iloc[1:2]
    # Get row matching TEAM_ABBREVIATION and GAME_ID from season_box
    match_row_one = season_box[(season_box['TEAM_ABBREVIATION'] == team_one) & (
        season_box['GAME_ID'] == id)]
    match_row_two = season_box[(season_box['TEAM_ABBREVIATION'] == team_two) & (
        season_box['GAME_ID'] == id)]
    # Reset index
    match_row_one.reset_index(drop=True, inplace=True)
    match_row_two.reset_index(drop=True, inplace=True)
    df_team_one.reset_index(drop=True, inplace=True)
    df_team_two.reset_index(drop=True, inplace=True)
    # Concat match_row with df_teams and
    df_match_row_one = pd.concat([match_row_one, df_team_one], axis=1)
    df_match_row_two = pd.concat([match_row_two, df_team_two], axis=1)
    # Append string '_1' to columns of df_match_row_two
    df_match_row_two = df_match_row_two.add_suffix('_1')
    # Append df_match_row_two to df_match_row_one as new columns
    df_one_line = pd.concat([df_match_row_one, df_match_row_two], axis=1)
    # Concat to season_box_ext
    season_box_ext = pd.concat([season_box_ext, df_one_line])

In [48]:
# Drop TEAM_ABBREVIATION column
season_box_ext.drop(['TEAM_ABBREVIATION'], axis=1, inplace=True)
# write season_box_ext to csv
season_box_ext = season_box_ext.to_csv('season_box_ext_20182019_one_line.csv', index=False)

AttributeError: 'NoneType' object has no attribute 'drop'